In [18]:
!pip install statsmodels

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# Report

In [19]:
from IPython.display import IFrame
IFrame("report.pdf", width=1000, height=600)

# Required Libraries

In [15]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Loading Data

In [16]:
df_sales = pd.read_csv("sales.txt", sep=";")
df_article = pd.read_csv("article_attributes.txt", sep=";")

FileNotFoundError: [Errno 2] File b'sales.txt' does not exist: b'sales.txt'

In [ ]:
display(df_sales.head())
display(df_article.head())

display(df_sales.describe())
display(df_article.describe())

### Merging Files

In [ ]:
df_all = pd.merge(df_sales, df_article, left_on='article', right_on='article',how='left')

display(df_all.head())
display(df_all.describe())

In [ ]:
col = df_all.columns.tolist()
col.remove("retailweek")

date_idx = pd.DatetimeIndex(df_all.retailweek.values.tolist(), name="date")

df_all_ts = pd.DataFrame(data=df_all[col].values.tolist(), index=date_idx, columns=col)
df_all_ts.sort_index(inplace=True)

countries = df_all_ts.country.unique()
display(countries)

display(df_all_ts.head())
display(df_all_ts.describe())

display(df_all_ts.dtypes)

In [ ]:
def encode(x):
    if x.dtype is np.dtype('O') and x.name != 'country':
        return x.astype('category').cat.codes
    return x

def normalize(df):
    return (df - df.mean()) / (df.max() - df.min())

### Correlations of Features

In [ ]:
df_encoded = df_all_ts.apply(encode)
df_encoded.groupby(['country']).corr()
fig, axes = plt.subplots(1, len(countries), figsize=(20, 5))
for i, c in enumerate(countries):
    ax = sns.heatmap(df_encoded.groupby(['country']).corr().xs(c), linewidths=1, ax = axes[i], square=True)
    ax.set_title(c)

### Selecting Specific Features by 'groupby'

In [7]:
df_promo12_sales_by_country_date = df_all_ts.groupby(['country', 'date'])[["promo1", "promo2", "sales"]].sum()
display(df_promo12_sales_by_country_date)

df_sales_by_country_date = df_promo12_sales_by_country_date['sales']
idx = df_sales_by_country_date.xs(countries[0]).index

NameError: name 'df_all_ts' is not defined

### Effects of Promos on Sales 

In [ ]:
df_norm = normalize(df_promo12_sales_by_country_date)

fig, axes = plt.subplots(1, len(countries), figsize=(30, 5))
for i, c in enumerate(countries):
    df_norm.xs(c).plot(title = c, ax = axes[i])

In [ ]:
fig, axes = plt.subplots(1, len(countries), figsize=(15, 3))
for i, c in enumerate(countries):
    ax = sns.heatmap(df_norm.xs(c).corr(), linewidths=1, ax = axes[i], square=True)
    ax.set_title(c)

In [ ]:
fmt_promo1 = '{}_promo1'
fmt_promo2 = '{}_promo2'

df_tmp = pd.DataFrame(index=idx)

df_promo1_by_date_country= df_promo12_sales_by_country_date['promo1']
df_promo2_by_date_country = df_promo12_sales_by_country_date['promo2']
for i, c in enumerate(countries):
    
    ncol = fmt_promo1.format(c)
    df_tmp[ncol] = df_promo1_by_date_country.xs(c)
    
    ncol = fmt_promo2.format(c)
    df_tmp[ncol] = df_promo2_by_date_country.xs(c)

figsize = (30,5)
df_tmp.plot.bar(figsize = figsize, stacked=True)

In [ ]:
sns.heatmap(df_tmp.corr(), linewidths=1, square=True)

# Time Series Analysis - Decomposition

In [ ]:
df_obs = pd.DataFrame(index=idx)
df_tre = pd.DataFrame(index=idx)
df_ses = pd.DataFrame(index=idx)
df_res = pd.DataFrame(index=idx)

decompositions = {}
components = ["Observed", "Trend", "Seasonal", "Residual"]

df_sales_by_country_date = df_promo12_sales_by_country_date['sales']

freq = 7
for i, c in enumerate(countries):
    decomposition = sm.tsa.seasonal_decompose(df_sales_by_country_date.xs(c), model='additive', freq = freq)
    df_obs[c] = decomposition.observed
    df_tre[c] = decomposition.trend
    df_ses[c] = decomposition.seasonal
    df_res[c] = decomposition.resid
    
    decompositions[components[0]] = df_obs
    decompositions[components[1]] = df_tre
    decompositions[components[2]] = df_ses
    decompositions[components[3]] = df_res
    
figsize = (15,5)
for i, d in enumerate(components):
    df = decompositions[d]
    df.plot(grid = True, title = d, figsize = figsize)

### Comparing Residual Components with Promos

In [ ]:
fmt_res = '{}_residual'

fig, axes = plt.subplots(1, len(countries), figsize=(30, 5))
for i, c in enumerate(countries):

    df_tmp = pd.DataFrame(index=idx)
    df_res = decompositions['Residual']
    df_tmp[fmt_res.format(c)] = df_res[c]
    df_tmp[fmt_promo1.format(c)] = df_promo1_by_date_country.xs(c)
    df_tmp[fmt_promo2.format(c)] = df_promo2_by_date_country.xs(c)
    df_tmp = normalize(df_tmp)
    df_tmp.plot(title = fmt_res.format(c), ax = axes[i])

In [ ]:
fig, axes = plt.subplots(1, len(decompositions), sharex=True, sharey=True, figsize=(16, 4))

for i, d in enumerate(components):
    df = decompositions[d]
    ax = sns.heatmap(df.corr(), annot=True, fmt=".2f", linewidths=1, ax = axes[i], square=True)
    ax.set_title(d)

### Final Checking of Residual Component by Autocorrelation, Histogram and Boxplot

In [ ]:
# df_res['day'] = list(map(lambda x: x.day, df_res.index.date))
# df_res['month'] = list(map(lambda x: x.month, df_res.index.date))
# df_res['year'] = list(map(lambda x: x.year, df_res.index.date))

lags = 10
fig, axes = plt.subplots(3, 4, figsize=(20, 20))

for i, c in enumerate(countries):
    
    df_res = decompositions['Residual']
    
    sm.graphics.tsa.plot_acf(df_res[c].values.tolist(), lags = lags,  ax = axes[i, 0])
    sm.graphics.tsa.plot_pacf(df_res[c].values.tolist(), lags = lags,  ax = axes[i, 1])
    
    df_res[c].hist(ax = axes[i, 2])

    df_tmp = pd.DataFrame(index=idx)
    df_tmp[c] = df_res[c]
    df_tmp['month'] =  list(map(lambda x: x.month, df_res.index.date))
    df_tmp.boxplot(column = c , by = 'month', ax = axes[i, 3])
    
plt.suptitle("")

In [ ]:
for i, c in enumerate(countries):
    df_res = decompositions['Residual']
    
    df_tmp = pd.DataFrame(index=idx)
    df_tmp[c] = df_res[c]
    df_tmp['month'] =  list(map(lambda x: x.month, df_res.index.date))

    fig, axes = plt.subplots(nrows=1, ncols=12, sharex=True, sharey=True, figsize=(24, 2))
    df_tmp.hist(column = c, by='month', ax=axes)
    fig.text(0.5, 1.1, c, ha='center')
    fig.text(0.04, 0.5, 'sales', va='center', rotation='vertical')
    #     plt.suptitle('Your Title Here', x=0.5, y=1.03, ha='center', fontsize='xx-large')


### Applying Moving Average, Moving Std , Autocorrelation, Partial Autocorrelation and Dickey-Fuller test

In [ ]:
# from pandas.plotting import autocorrelation_plot
#     autocorrelation_plot(df_selected_fields.xs(c), ax = axes[i, 1])

from statsmodels.tsa.stattools import adfuller

df_sales_by_country_date.sort_index(inplace=True)

fmt_sales = "{}_Sales"
fmt_mean = "{}_Rolling_Mean"
fmt_std = "{}_Rolling_Std"

window = 12
lags = 7

f, axes = plt.subplots(len(countries), 3, figsize=(20, 20)) 
for i, c in enumerate(countries):
    df_tmp = pd.DataFrame(index=idx)

    ncol = fmt_sales.format(c)
    df_tmp[ncol] = df_sales_by_country_date.xs(c)
    
    ncol = fmt_mean.format(c)
    df_mean = df_sales_by_country_date.xs(c).rolling(window = window, center = False).mean()
    df_tmp[ncol] = df_mean
    
    ncol = fmt_std.format(c)
    df_std = df_sales_by_country_date.xs(c).rolling(window = window, center = False).std()
    df_tmp[ncol] = df_std
    
    df_tmp.plot(title = c, ax = axes[i, 0])
    sm.graphics.tsa.plot_acf(df_sales_by_country_date.xs(c), lags = lags,  ax = axes[i, 1])
    sm.graphics.tsa.plot_pacf(df_sales_by_country_date.xs(c), lags = lags,  ax = axes[i, 2])
    
#Perform Dickey-Fuller test:
for i, c in enumerate(countries):
    print('Results of Dickey-Fuller Test [{}]:'.format(c))
    dftest = adfuller(df_sales_by_country_date.xs(c).values, autolag='AIC', maxlag = lags)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value ({})'.format(key)] = value
    display(dfoutput)
    

# Building Models for Each Country by SARIMAX

In [ ]:
models = {}

for i, c in enumerate(countries):
    mod = sm.tsa.statespace.SARIMAX(df_sales_by_country_date.xs(c), trend='ct', order=(4,1,0), seasonal_order=(0,1,1,7))
    results = mod.fit()
    models[c] = results
    display(results.summary())
    
fig, axes = plt.subplots(len(countries), 1) 
for i, c in enumerate(countries):
    results = models[c]
    r = pd.DataFrame(index=idx)
    r[c] = df_sales_by_country_date.xs(c)
    pred = models[c].get_prediction(start=0)
    pred_ci = pred.conf_int()
    r['Forecast_' + c] = pred.predicted_mean
    ax = axes[i]
    r.plot(figsize=(10, 15), ax = ax, sharey=True, alpha=.7)
    ax.fill_between(pred_ci.index, pred_ci.iloc[:, 0], pred_ci.iloc[:, 1], color='k', alpha=.1)

    plt.legend()
    sns.despine()